In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit import Aer

In [ ]:
import numpy as np

# 1) Dataset
X, y = datasets.make_moons(n_samples=500, noise=0.2, random_state=42)

# Normalizar
X = StandardScaler().fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2) Kernels clásicos
kernels = {
    "Lineal": SVC(kernel="linear"),
    "RBF": SVC(kernel="rbf", gamma='scale'),
    "Polynomial": SVC(kernel="poly", degree=3)
}

print("Entrenando modelos clásicos...\n")

for name, model in kernels.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{name}: {acc:.3f}")

# 3) Quantum Kernel
print("\nEntrenando modelo con Quantum Kernel...\n")

# FeatureMap: ZZFeatureMap
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)

# Backend (statevector is perfect for simulación ideal)
backend = Aer.get_backend('statevector_simulator')

quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

# SVM con kernel cuántico
svc_q = SVC(kernel=quantum_kernel.evaluate)

svc_q.fit(X_train, y_train)
preds_q = svc_q.predict(X_test)
acc_q = accuracy_score(y_test, preds_q)

print(f"Quantum Kernel (ZZFeatureMap): {acc_q:.3f}")
